In [2]:
#Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
path = r'/Users/prateeksharma/Documents/Instacart Basket Analysis'

In [4]:
#Importing data set
df_final=pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_viz_Part1.pkl'))

In [5]:
#Looking for PII data
df_final.dtypes

product_id                   int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
order_id                     int64
user_id                      int64
order_number                 int64
order_dow                    int64
order_hour_of_day            int64
days_since_prior_order     float64
add_to_cart_order            int64
reordered                    int64
Busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
spending_habits            float64
spending_flag               object
order_frequency            float64
order_frequency_flag        object
first_name                  object
last_name                   object
gender                      object
state                       object
age                          int64
date_joined                 object
number_of_dependants         int64
marital_status              object
income              

## Addressing PII 
user_id,first_name,last_name,gender,state,age,date_joined all of these can be considered as PII data. Replacing or removing either of the two or both ; first_name,last_name   can be done to protect individual identities or reduce the risk of it .Additionaly Scrubadub package can be used (totally based on how an organisation deals with PII) 

In [6]:
#Creating Regional flag for Northeast Region
df_final.loc[df_final['state'].isin(['Maine','New Hampshire','Vermont','Massachusetts', 'Rhode Island',
                                     'Connecticut','New York','Pennsylvania','New Jersey']),
                                     'regional_flag'] = 'Northeast'

In [7]:
#Creating Regional flag for Midwest Region
df_final.loc[df_final['state'].isin(['Wisconsin','Michigan','Illinois','Indiana', 'Ohio', 'North Dakota',
                                     'South Dakota','Nebraska','Kansas','Minnesota','Iowa','Missouri']),
                                     'regional_flag'] = 'Midwest'

In [8]:
#Creating Regional flag for South Region
df_final.loc[df_final['state'].isin(['Delaware','Maryland','District of Columbia','Virginia', 'West Virginia',
                                     'North Carolina','South Carolina','Georgia','Florida','Kentucky','Tennessee',
                                     'Mississippi','Alabama','Oklahoma','Texas','Arkansas','Louisiana']),
                                     'regional_flag'] = 'South'

In [9]:
#Creating Regional flag for West Region
df_final.loc[df_final['state'].isin(['Idaho','Montana','Wyoming','Nevada','Utah','Colorado','Arizona','New Mexico',
                                     'Alaska','Washington','Oregon','California','Hawaii']),'regional_flag'] = 'West'

In [11]:
#Checking value counts
df_final['regional_flag'].value_counts(dropna=False)

regional_flag
South        10099817
West          7765088
Midwest       7107586
Northeast     5356272
Name: count, dtype: int64

In [11]:
#Creating crosstable to determine the spending habits among different U.S regions
crosstab_region = pd.crosstab(df_final['regional_flag'], df_final['spending_flag'], dropna=False)

In [12]:
#Checking output
crosstab_region

spending_flag,High spender,Low spender
regional_flag,,
Midwest,148321,6959265
Northeast,102905,5253367
South,197110,9902707
West,149922,7615166


In [13]:
#Creating an exclusion flag for low-activity customers (< 5 orders)
df_final['max_order'] = df_final.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/1h/3sdj2tc1297dytyhjybm86l00000gn/T/ipykernel_3994/2214436393.py:2: FutureWarning: The provided callable <function amax at 0x10e3ffec0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_final['max_order'] = df_final.groupby(['user_id'])['order_number'].transform(np.max)


In [23]:
#Creating an empty list for low activity and active customers using If-statements with For-loop
result = []

for value in df_final["max_order"]:
    if value < 5:
        result.append("low_activiity")
    else:
        result.append("active")

In [25]:
#Creating a new column 'customer_activity'within df_final and setting it equal to result
df_final['customer_activity']= result

In [26]:
df_final['customer_activity'].value_counts(dropna=False)

customer_activity
active           29303281
low_activiity     1025482
Name: count, dtype: int64

In [33]:
#Subsetting active customers
df_final_active=df_final[df_final['customer_activity']=='active']

In [34]:
df_final_active.shape

(29303281, 32)

In [35]:
#Exporting active customers list
df_final_active.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_active_cust.pkl'))